In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# List of features
def model_fn(features, labels, mode):
    # Build linear model
    W = tf.get_variable("W", [1], dtype=tf.float64)
    b = tf.get_variable("b", [1], dtype=tf.float64)
    y = W*features['x'] + b
    
    # Loss sub-graph
    loss = tf.reduce_sum(tf.square(y - labels))
    
    # Training sub-graph
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = tf.group(optimizer.minimize(loss),
                    tf.assign_add(global_step, 1))
    
    # EstimatorSpec connects subgraphs we built to the appropriate functionality
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=y,
        loss=loss,
        train_op=train)

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define data sets
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7., 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# train
estimator.train(input_fn=input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x111c54390>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/var/folders/61/cwfb4pw94vj6q3s_y3zcssbm0000gn/T/tmpGs4a6q', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/61/cwfb4pw94vj6q3s_y3zcssbm0000gn/T/tmpGs4a6q/model.ckpt.
INFO:tensorflow:loss = 17.193012864, step = 1
INFO:tensorflow:global_step/sec: 744.841
INFO:tensorflow:loss = 0.231686763459, step = 101 (0.131 sec)
INFO:tensorflow:global_step/sec: 821.923
INFO:tensorflow:loss = 0.012932

In [3]:
# Evaluate how well model did
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r" % train_metrics)
print("eval metrics: %r" % eval_metrics)

INFO:tensorflow:Starting evaluation at 2018-01-13-05:03:21
INFO:tensorflow:Restoring parameters from /var/folders/61/cwfb4pw94vj6q3s_y3zcssbm0000gn/T/tmpGs4a6q/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-13-05:03:22
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 7.70107e-11
INFO:tensorflow:Starting evaluation at 2018-01-13-05:03:23
INFO:tensorflow:Restoring parameters from /var/folders/61/cwfb4pw94vj6q3s_y3zcssbm0000gn/T/tmpGs4a6q/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-01-13-05:03:24
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, loss = 0.0101016
train metrics: {'loss': 7.7010717e-11, 'global_step': 1000}
eval metrics: {'loss': 0.010101607, 'global_step': 1000}
